In [1]:
import sqlite3 as lite
import common_functions_cache as c
import pickle
from sqlalchemy import create_engine
import pandas as pd
import re

In [2]:
#pd.set_option('max_colwidth',2000)
pd.options.display.max_colwidth = 100000


In [3]:
conn = lite.connect('cluster_db')
cursor = conn.cursor()

In [4]:
print(c.__doc__)

Version of common functions with more caching, also using new sqlite db schema (LTCC_cache). Functions for querying abstracts on ePMC, saving results in sqlite db and displaying results in dataframe.
More checks in this version to avoid duplicate retrieval of information from web services. 
Functions available:
create_db(db_name)
pop_chembl_pmids(db_name) -- this populates the chembl_pmids table with pmids from a specific chembl_version.
def define_synonym_queries(term_dict_1, term_dict_2 = None, term_dict_3 = None)
get_hit_profile(query_list)
get_pmids(query, db_name) -- query is string
get_article_data(query_id, db_name) -- query is string
get_pmids_and_article_data(query, db_name) -- query is string
get_availabilities(query_id, db_name)
get_scores(query_id, db_name)
set_chembl_values(query_id, db_name)
get_df(query_id_list, db_name, sql_condition=None)
separate_column_df(query_id)
colour_terms(df, markup_list)
plot_scores(query_id_list, db_name)



In [5]:
# cursor.execute('select * from queries where query_id = 12')
# q = cursor.fetchall()[0][1]
# [x.strip().replace('\n', ' ') for x in q.split("OR") if 'ABSTRACT' in x]

In [7]:
include_terms = [x for x in updated_terms if x['name'] == 'interesting_tissue'][0]['terms']

exclude_terms = [x for x in updated_terms if x['name'] == 'unwanted_tissue'][0]['terms']

In [8]:
include_pat = re.compile(r'\b(?:' + '|'.join(include_terms) + r')\b', re.I)

exclude_pat = re.compile(r'\b(?:' + '|'.join(exclude_terms) + r')\b', re.I)

In [9]:
df = c.get_df([12],'cluster_db', sql_condition = '''
    a.in_chembl = 0
and al.request_access is null
''')[0]

df.shape

(2557, 14)

### Dataset counts

In [10]:
# dataset 1
df[df.abstract.str.contains(include_pat) & ~df.abstract.str.contains(exclude_pat)].shape

(372, 14)

In [11]:
# dataset 2
df[df.abstract.str.contains(include_pat) & df.abstract.str.contains(exclude_pat)].shape

(225, 14)

In [12]:
# dataset 3
df[~df.abstract.str.contains(include_pat) & ~df.abstract.str.contains(exclude_pat)].shape

(974, 14)

In [13]:
# dataset 4
df[~df.abstract.str.contains(include_pat) & df.abstract.str.contains(exclude_pat)].shape

(986, 14)

### View datasets

In [45]:
with open('terms_for_highlighting.pkl', 'rb') as f: 
    
    updated_terms = pickle.load(f) 

In [46]:
# Inspect dataset 1

df = df[df.abstract.str.contains(include_pat) & ~df.abstract.str.contains(exclude_pat)][['pmid_link', 'year', 'title', 'abstract', 'journal', 'pdf_links', 'other_links', 'campus_links']].reset_index(drop=True)

c.colour_terms(df, updated_terms)